<a href="https://colab.research.google.com/github/UnpackJungHo/XRSimulator_Osaka/blob/Learning_AI/LSTM_TEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from datetime import datetime, timedelta
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import warnings
warnings.filterwarnings('ignore')

class WeatherPredictor:
    def __init__(self):
        self.models = {}
        self.scalers = {}  # 각 변수별 스케일러
        self.label_encoders = {}
        self.sequence_length = 24  # 24시간의 데이터로 다음을 예측

    def load_and_preprocess_data(self, training=True):
        """데이터 로드 및 전처리"""
        try:
            if training:
                # 학습용 데이터 (2020-2022)
                df_2020 = pd.read_excel('2020_weather.xlsx', dtype={'DateTime(YYYYMMDDHHMI)': str})
                df_2021 = pd.read_excel('2021_weather.xlsx', dtype={'DateTime(YYYYMMDDHHMI)': str})
                df_2022 = pd.read_excel('2022_weather.xlsx', dtype={'DateTime(YYYYMMDDHHMI)': str})
                print("학습 데이터 로드 완료 (2020-2022)")
                df = pd.concat([df_2020, df_2021, df_2022], ignore_index=True)
            else:
                # 검증용 데이터 (2023)
                df = pd.read_excel('2023_weather.xlsx', dtype={'DateTime(YYYYMMDDHHMI)': str})
                print("검증 데이터 로드 완료 (2023)")

            # 전처리 수행
            df = self._basic_preprocessing(df)
            df = self._create_advanced_features(df)
            df = self._handle_outliers(df)
            df = self._handle_missing_values(df)

            return df

        except Exception as e:
            print("데이터 로드 중 오류 발생:")
            print(e)
            raise e

    def _basic_preprocessing(self, df):
        """기본 전처리 작업"""
        try:
            # DateTime 변환
            df['DateTime(YYYYMMDDHHMI)'] = pd.to_datetime(df['DateTime(YYYYMMDDHHMI)'],
                                                        format='%Y%m%d%H%M')

            # 시간 관련 특성 추가
            df['hour'] = df['DateTime(YYYYMMDDHHMI)'].dt.hour
            df['month'] = df['DateTime(YYYYMMDDHHMI)'].dt.month
            df['day'] = df['DateTime(YYYYMMDDHHMI)'].dt.day
            df['dayofweek'] = df['DateTime(YYYYMMDDHHMI)'].dt.dayofweek
            df['is_weekend'] = df['dayofweek'].isin([5, 6]).astype(int)

            # 순환 시간 특성
            df['hour_sin'] = np.sin(2 * np.pi * df['hour']/24)
            df['hour_cos'] = np.cos(2 * np.pi * df['hour']/24)
            df['month_sin'] = np.sin(2 * np.pi * df['month']/12)
            df['month_cos'] = np.cos(2 * np.pi * df['month']/12)

            # 범주형 변수 인코딩
            categorical_columns = ['WW', 'CT']
            for col in categorical_columns:
                if df[col].dtype == 'object':
                    if col not in self.label_encoders:
                        self.label_encoders[col] = LabelEncoder()
                        self.label_encoders[col].fit(df[col].astype(str))
                    else:
                        # 새로운 카테고리 처리
                        unique_labels = set(df[col].astype(str).unique())
                        known_labels = set(self.label_encoders[col].classes_)
                        new_labels = unique_labels - known_labels

                        if new_labels:
                            print(f"\n{col}에서 발견된 새로운 레이블: {new_labels}")
                            for new_label in new_labels:
                                df.loc[df[col].astype(str) == new_label, col] = self.label_encoders[col].classes_[0]

                    df[col] = self.label_encoders[col].transform(df[col].astype(str))

            return df

        except Exception as e:
            print("기본 전처리 중 오류 발생:")
            print(e)
            raise e

    def _create_advanced_features(self, df):
        """고급 특성 생성"""
        # 시계열 특성
        for col in ['TA', 'WS', 'HM', 'RN', 'WD']:
            # 이동평균
            df[f'{col}_MA1'] = df[col].rolling(window=1).mean()
            df[f'{col}_MA3'] = df[col].rolling(window=3).mean()
            df[f'{col}_MA6'] = df[col].rolling(window=6).mean()
            df[f'{col}_MA12'] = df[col].rolling(window=12).mean()

            # 변화율
            df[f'{col}_change'] = df[col].diff()
            df[f'{col}_change_rate'] = df[col].pct_change()

            # 시간별 통계
            df[f'{col}_hour_mean'] = df.groupby('hour')[col].transform('mean')
            df[f'{col}_hour_std'] = df.groupby('hour')[col].transform('std')

        return df

    def _handle_outliers(self, df):
        """이상치 처리"""
        numerical_columns = ['TA', 'WS', 'HM', 'RN']
        for col in numerical_columns:
            mean = df[col].mean()
            std = df[col].std()
            df = df[np.abs(df[col] - mean) <= 3 * std]
        return df

    def _handle_missing_values(self, df):
        """결측치 처리"""
        numerical_columns = ['TA', 'WS', 'HM', 'RN']
        for col in numerical_columns:
            df[col].fillna(df[col].rolling(window=6, min_periods=1).mean(), inplace=True)
        df.fillna(-999, inplace=True)
        return df


    def create_sequences(self, data, target_col):
        """시계열 시퀀스 생성"""
        X, y = [], []
        for i in range(len(data) - self.sequence_length):
            X.append(data[i:(i + self.sequence_length)])
            y.append(data[target_col].iloc[i + self.sequence_length])
        return np.array(X), np.array(y)

    def create_lstm_model(self, input_shape):
        """LSTM 모델 생성"""
        model = Sequential([
            LSTM(64, return_sequences=True, input_shape=input_shape),
            Dropout(0.2),
            LSTM(32),
            Dropout(0.2),
            Dense(16, activation='relu'),
            Dense(1)
        ])
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
        return model

    def train_models(self, df):
      """모델 학습"""
      # 예측할 변수들
      target_columns = ['TA', 'RN', 'WS', 'HM', 'WD']

      # 특성 선택
      feature_columns = [col for col in df.columns
                        if col not in ['DateTime(YYYYMMDDHHMI)'] + target_columns]

      for target in target_columns:
          print(f"\n{target} 모델 학습 중...")

          # 데이터 준비
          data = df[feature_columns + [target]].copy()

          # 무한대 값 처리
          data = data.replace([np.inf, -np.inf], np.nan)

          # 이상치 처리
          for col in data.columns:
              if data[col].dtype in ['int64', 'float64']:
                  Q1 = data[col].quantile(0.25)
                  Q3 = data[col].quantile(0.75)
                  IQR = Q3 - Q1
                  lower_bound = Q1 - 1.5 * IQR
                  upper_bound = Q3 + 1.5 * IQR
                  data[col] = data[col].clip(lower_bound, upper_bound)

          # 결측치 처리
          data = data.fillna(data.mean())

          # 데이터 스케일링
          scaler = MinMaxScaler()
          scaled_data = scaler.fit_transform(data)
          scaled_df = pd.DataFrame(scaled_data, columns=data.columns)
          self.scalers[target] = scaler

          # 시퀀스 생성
          X, y = self.create_sequences(scaled_df, target)

          # 학습/검증 분할
          train_size = int(len(X) * 0.8)
          X_train, X_test = X[:train_size], X[train_size:]
          y_train, y_test = y[:train_size], y[train_size:]

          # 모델 생성 및 학습
          model = self.create_lstm_model((self.sequence_length, X.shape[2]))

          # Early Stopping 추가
          early_stopping = tf.keras.callbacks.EarlyStopping(
              monitor='val_loss',
              patience=5,
              restore_best_weights=True
          )

          # 모델 학습
          history = model.fit(
              X_train, y_train,
              epochs=50,
              batch_size=32,
              validation_split=0.2,
              callbacks=[early_stopping],
              verbose=1
          )

          self.models[target] = model

          # 성능 평가
          y_pred = model.predict(X_test)
          mse = mean_squared_error(y_test, y_pred)
          r2 = r2_score(y_test, y_pred)

          print(f"{target} 모델 성능:")
          print(f"MSE: {mse:.4f}")
          print(f"R2 Score: {r2:.4f}")

    def predict_and_evaluate(self, input_time, validation_data):
      """예측 수행 및 평가"""
      input_datetime = pd.to_datetime(input_time, format='%Y/%m/%d/%H:%M')

      # 입력 시퀀스 준비
      past_data = validation_data[
          validation_data['DateTime(YYYYMMDDHHMI)'] <= input_datetime
      ].tail(self.sequence_length)

      if len(past_data) < self.sequence_length:
          print(f"예측을 위해 최소 {self.sequence_length}시간의 데이터가 필요합니다.")
          return None

      predictions = []
      current_sequence = past_data.copy()

      # 예측할 변수들
      target_columns = ['TA', 'RN', 'WS', 'HM', 'WD']
      feature_columns = [col for col in current_sequence.columns
                        if col not in ['DateTime(YYYYMMDDHHMI)'] + target_columns]

      for hour in range(6):
          next_time = input_datetime + timedelta(hours=hour)

          hour_prediction = {
              'DateTime': next_time
          }

          # 각 타겟 변수에 대한 예측
          for target in target_columns:
              # 입력 데이터 준비
              data = current_sequence[feature_columns + [target]].copy()

              # 무한대 값 처리
              data = data.replace([np.inf, -np.inf], np.nan)

              # 결측치 처리
              data = data.fillna(data.mean())

              # 스케일링
              scaler = self.scalers[target]
              scaled_sequence = scaler.transform(data)

              # 예측
              X = scaled_sequence.reshape(1, self.sequence_length, -1)
              scaled_pred = self.models[target].predict(X, verbose=0)[0]

              # 역스케일링을 위한 더미 데이터 생성
              dummy_data = np.zeros((1, data.shape[1]))
              dummy_data[0, -1] = scaled_pred
              pred = scaler.inverse_transform(dummy_data)[0, -1]

              hour_prediction[f'{target}_pred'] = pred

              # 실제값 찾기
              actual_row = validation_data[
                  validation_data['DateTime(YYYYMMDDHHMI)'] == next_time
              ]
              if not actual_row.empty:
                  hour_prediction[f'{target}_actual'] = actual_row[target].iloc[0]
              else:
                  hour_prediction[f'{target}_actual'] = None

          predictions.append(hour_prediction)

          # 시퀀스 업데이트
          new_row = current_sequence.iloc[-1:].copy()
          for target in target_columns:
              new_row[target] = hour_prediction[f'{target}_pred']
          current_sequence = pd.concat([current_sequence[1:], new_row])

      results_df = pd.DataFrame(predictions)

      # 예측 성능 평가
      print("\n예측 성능 평가:")
      for target in target_columns:
          mask = results_df[f'{target}_actual'].notna()
          if mask.any():
              mae = mean_absolute_error(
                  results_df[mask][f'{target}_actual'],
                  results_df[mask][f'{target}_pred']
              )
              mse = mean_squared_error(
                  results_df[mask][f'{target}_actual'],
                  results_df[mask][f'{target}_pred']
              )
              r2 = r2_score(
                  results_df[mask][f'{target}_actual'],
                  results_df[mask][f'{target}_pred']
              )

              print(f"\n{target} 예측 성능:")
              print(f"MAE: {mae:.4f}")
              print(f"MSE: {mse:.4f}")
              print(f"R2 Score: {r2:.4f}")

      return results_df

In [35]:
# 예측기 인스턴스 생성
predictor = WeatherPredictor()

# 학습 데이터 로드 및 전처리
print("학습 데이터 로드 및 전처리 중...")
train_df = predictor.load_and_preprocess_data(training=True)

# 검증 데이터 로드
print("\n검증 데이터 로드 중...")
validation_df = predictor.load_and_preprocess_data(training=False)

학습 데이터 로드 및 전처리 중...
학습 데이터 로드 완료 (2020-2022)

검증 데이터 로드 중...
검증 데이터 로드 완료 (2023)

WW에서 발견된 새로운 레이블: {'4240', '19060201', '601', '190602', '1602', '160201', '19060502', '190501', '400601', '1601'}

CT에서 발견된 새로운 레이블: {'CuAcCi', 'CuCs'}


In [36]:
# 모델 학습
print("\n모델 학습 시작...")
predictor.train_models(train_df)


모델 학습 시작...

TA 모델 학습 중...
Epoch 1/50
514/514 ━━━━━━━━━━━━━━━━━━━━ 21s 32ms/step - loss: 0.0460 - val_loss: 0.0012
Epoch 2/50
514/514 ━━━━━━━━━━━━━━━━━━━━ 18s 28ms/step - loss: 0.0031 - val_loss: 0.0025
Epoch 3/50
514/514 ━━━━━━━━━━━━━━━━━━━━ 20s 28ms/step - loss: 0.0020 - val_loss: 0.0016
Epoch 4/50
514/514 ━━━━━━━━━━━━━━━━━━━━ 14s 28ms/step - loss: 0.0014 - val_loss: 9.7690e-04
Epoch 5/50
514/514 ━━━━━━━━━━━━━━━━━━━━ 21s 28ms/step - loss: 9.9231e-04 - val_loss: 5.9447e-04
Epoch 6/50
514/514 ━━━━━━━━━━━━━━━━━━━━ 21s 28ms/step - loss: 7.9336e-04 - val_loss: 9.8357e-04
Epoch 7/50
514/514 ━━━━━━━━━━━━━━━━━━━━ 14s 28ms/step - loss: 6.7228e-04 - val_loss: 4.8883e-04
Epoch 8/50
514/514 ━━━━━━━━━━━━━━━━━━━━ 14s 28ms/step - loss: 6.1067e-04 - val_loss: 3.4674e-04
Epoch 9/50
514/514 ━━━━━━━━━━━━━━━━━━━━ 20s 27ms/step - loss: 5.1078e-04 - val_loss: 3.2455e-04
Epoch 10/50
514/514 ━━━━━━━━━━━━━━━━━━━━ 14s 28ms/step - loss: 4.7558e-04 - val_loss: 3.3937e-04
Epoch 11/50
514/514 ━━━━━━━━━━━━━━━━━━━

In [ ]:
# 예측 루프
while True:
    # 사용자로부터 날짜 입력 받기
    input_time = input("\n예측할 날짜와 시간을 입력하세요 (형식: YYYY/MM/DD/HH:MM, 종료는 'q'): ")

    if input_time.lower() == 'q':
        break

    try:
        # 예측 수행 및 평가
        results = predictor.predict_and_evaluate(input_time, validation_df)

        if results is not None:
            print("\n예측 결과:")
            pd.set_option('display.max_columns', None)
            print(results[['DateTime'] +
                        [col for col in results.columns if 'pred' in col or 'actual' in col]])

    except Exception as e:
        print(f"오류 발생: {str(e)}")
        print("올바른 형식으로 다시 입력해주세요.")


예측할 날짜와 시간을 입력하세요 (형식: YYYY/MM/DD/HH:MM, 종료는 'q'): 2023/06/13/13:00

예측 성능 평가:

TA 예측 성능:
MAE: 4.0697
MSE: 16.7179
R2 Score: -18.3333

RN 예측 성능:
MAE: 0.0013
MSE: 0.0000
R2 Score: 0.0000

WS 예측 성능:
MAE: 0.6848
MSE: 0.6243
R2 Score: -2.6724

HM 예측 성능:
MAE: 9.3226
MSE: 97.7956
R2 Score: -22.1621

WD 예측 성능:
MAE: 3.2162
MSE: 11.7791
R2 Score: -7.8343

예측 결과:
             DateTime    TA_pred  TA_actual   RN_pred  RN_actual   WS_pred  \
0 2023-06-13 13:00:00  22.184973       25.6 -8.995197       -9.0  3.098354   
1 2023-06-13 14:00:00  22.023750       25.9 -8.997653       -9.0  3.138268   
2 2023-06-13 15:00:00  21.420579       25.7 -8.999654       -9.0  3.142420   
3 2023-06-13 16:00:00  20.682279       25.4 -9.000007       -9.0  3.124377   
4 2023-06-13 17:00:00  19.992436       24.1 -9.000007       -9.0  3.105080   
5 2023-06-13 18:00:00  19.377869       23.4 -9.000008       -9.0  3.082682   

   WS_actual    HM_pred  HM_actual    WD_pred  WD_actual  
0        3.3  53.253717         58  2